In [4]:
# -----------------------------
# 1. Imports
# -----------------------------
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm

# -----------------------------
# 2. Paths
# -----------------------------
data_dir = "/kaggle/input/skin-cancer-mnist-ham10000"
metadata_path = os.path.join(data_dir, "HAM10000_metadata.csv")
image_folders = [
    os.path.join(data_dir, "HAM10000_images_part_1"),
    os.path.join(data_dir, "HAM10000_images_part_2")
]
print("Files in data directory:", os.listdir(data_dir))

# -----------------------------
# 3. Load Dataset
# -----------------------------
def load_dataset(metadata_path, test_size=0.2, random_state=42):
    labels_df = pd.read_csv(metadata_path)
    label_map = {cls:i for i, cls in enumerate(labels_df['dx'].unique())}
    labels_df['label'] = labels_df['dx'].map(label_map)
    train_df, test_df = train_test_split(labels_df, test_size=test_size, stratify=labels_df['label'], random_state=random_state)
    return train_df, test_df, label_map

train_df, test_df, label_map = load_dataset(metadata_path)
print(f"Number of classes: {len(label_map)}")

# -----------------------------
# 4. Dataset Class
# -----------------------------
class HAM10000Dataset(Dataset):
    def __init__(self, df, data_dirs, transform=None):
        self.df = df.reset_index(drop=True)
        if isinstance(data_dirs, str):
            self.data_dirs = [data_dirs]
        else:
            self.data_dirs = data_dirs
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'image_id'] + '.jpg'
        img_path = None
        for folder in self.data_dirs:
            candidate = os.path.join(folder, img_name)
            if os.path.exists(candidate):
                img_path = candidate
                break
        if img_path is None:
            raise FileNotFoundError(f"{img_name} not found in any folder")
        image = Image.open(img_path).convert('RGB')
        label = self.df.loc[idx, 'label']
        if self.transform:
            image = self.transform(image)
        return image, label

# -----------------------------
# 5. Transformations
# -----------------------------
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = HAM10000Dataset(train_df, data_dirs=image_folders, transform=train_transform)
test_dataset = HAM10000Dataset(test_df, data_dirs=image_folders, transform=test_transform)

# -----------------------------
# 6. DataLoaders
# -----------------------------
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# -----------------------------
# 7. Train T-ResNet50 Classifier
# -----------------------------
def train_classifier(train_loader, n_classes, device='cuda', n_epochs=20):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 128),
        nn.ReLU(),
        nn.Linear(128, n_classes)
    )
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0
        for imgs, labels in tqdm(train_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{n_epochs}] | Loss: {running_loss/len(train_loader):.4f}")
    return model

n_classes = len(label_map)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = train_classifier(train_loader, n_classes, device=device, n_epochs=20)

# -----------------------------
# 8. Evaluation
# -----------------------------
def evaluate_model(model, loader, device='cuda'):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro')
    rec = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
    print("Confusion Matrix:\n", cm)

evaluate_model(model, test_loader, device=device)


Files in data directory: ['hmnist_8_8_RGB.csv', 'hmnist_28_28_RGB.csv', 'HAM10000_images_part_1', 'ham10000_images_part_1', 'hmnist_8_8_L.csv', 'HAM10000_images_part_2', 'ham10000_images_part_2', 'hmnist_28_28_L.csv', 'HAM10000_metadata.csv']
Number of classes: 7


100%|██████████| 251/251 [01:55<00:00,  2.17it/s]


Epoch [1/20] | Loss: 0.7155


100%|██████████| 251/251 [01:56<00:00,  2.15it/s]


Epoch [2/20] | Loss: 0.5120


...

Epoch [18/20] | Loss: 0.0741


100%|██████████| 251/251 [01:55<00:00,  2.18it/s]


Epoch [19/20] | Loss: 0.0739


100%|██████████| 251/251 [01:54<00:00,  2.19it/s]


Epoch [20/20] | Loss: 0.0743
Accuracy: 0.8857, Precision: 0.8310, Recall: 0.7422, F1: 0.7811
Confusion Matrix:
 [[ 172   24    0   15    0    2    7]
 [  16 1296    1   23    0    5    0]
 [   1    6   14    1    0    1    0]
 [   7   64    1  148    0    2    1]
 [   0    3    0    1   22    2    0]
 [   2    8    1    2    0   86    4]
 [   9    6    2    5    0    7   36]]
